In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
# spark_version = 'spark-3.<spark version>'
os.environ['SPARK_VERSION']=spark_version


# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-06-09 04:45:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.69MB/s    in 0.2s    

2022-06-09 04:45:58 (4.69 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Pet Productdataset
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [ ]:
#Count initial records in dataset
print(f'There are {df.count()} records')

There are 2643619 records


In [ ]:
#Drop NA and count records in dataset
df = df.dropna()
print(f'There are {df.count()} records after dropping NAs')

There are 2643241 records after dropping NAs


In [ ]:
#Drop duplicates and count records in dataset
df = df.dropDuplicates()
print(f'There are {df.count()} records after dropping duplicates')

There are 2643241 records after dropping duplicates


In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [44]:
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType,LongType,DateType

In [51]:
review_id_table_df = df.select(['review_id','customer_id','product_id','product_parent','review_date'])

In [53]:
review_id_table_df.withColumn("review_id", review_id_table_df.review_id.cast(StringType())).withColumn("customer_id", review_id_table_df.customer_id.cast(StringType())).withColumn("product_id", review_id_table_df.product_id.cast(StringType())).withColumn("product_parent", review_id_table_df.product_parent.cast(StringType())).withColumn("review_date", review_id_table_df.review_date.cast(StringType()))

DataFrame[review_id: string, customer_id: string, product_id: string, product_parent: string, review_date: string]

In [41]:
review_id_table_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100EO2AQP6B02|   33140435|B00BUQ4AV8|     805940477| 2014-10-12|
|R100IITU077KKZ|   25475634|B00RVY9DRO|     189955978| 2015-06-04|
|R100WIAVSCW4D1|   49229898|B004M8LM44|     419962202| 2013-07-31|
|R1023NXBAJ0BYT|   52227830|B0006IZH68|     470609741| 2004-12-23|
|R102CEVVIBXNFB|   11062585|B0014N70QO|     226028756| 2014-07-14|
|R102MO11LN79HV|   17155412|B0039LFVUG|     517238077| 2010-07-24|
|R102TPQOB6ECI4|   46107297|B008QVKBLS|     212972268| 2015-02-27|
|R1032IWOCCKRV1|   28307907|B00406YHRC|     178991753| 2014-01-08|
|R103QNPJX6OJ6D|   38188681|B003TUQWB4|     287424702| 2014-05-27|
|R104GDOC82SGT4|   49476810|B000YIWUXI|     178342731| 2015-03-13|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [ ]:
products_df = df.select(['product_id','product_title'])


In [ ]:
products_df = products_df.dropDuplicates()

In [ ]:
products_df.count()

239307

In [28]:
tempdf = df.select(['customer_id'])

In [29]:
tempdf2 = tempdf.groupby('customer_id').agg({'customer_id':'count'})

In [31]:
customers_df = tempdf2.withColumnRenamed("count(customer_id)", "customer_count")

In [43]:
customers_df.count()

1414974

In [56]:
review_id_table_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [48]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)